## Make Production Run List

__Description__: Calculates the discharge from the mean flow frequency curve for each annual exceedance probability and corresponding weight.

__Input__: The mean flow frequency curve table, and the annual exceedance probability with corresponding weights table.

__Output__: A table with the annual exceedance probability, discharge, and weights.


---

#### Load libraries and Python options:

In [1]:
import sys
sys.path.append('../../core')
from nb_libraries import*
from fluvial_hydro import*

#### Papermill parameters:

#### Initalize a table to store the calculations:

In [3]:
AEP_Weights = pd.DataFrame.from_dict(AEP_Weights) #Convert from a dictionary to a pandas dataframe

AEP = AEP_Weights['AEP'] #Extract the annual exceedance probability column whose discharge is to be calculated

AEPz = zvar(AEP) #Convert to the normal z variate

table = ffc_summary(AEP, AEPz) #Initalize a table with AEP and AEPz

table['RI'] = 1.0/table.index #Add a column to the table which is the recurrance interval

table.head(2)

,AEPz,RI
AEP,,
0.482369,0.044208,2.073100
0.448352,0.129827,2.230391


#### Extract the data from the mean flow frequency curve table:

In [4]:
mean_curve = pd.DataFrame.from_dict(mean_curve) #Convert from a dictionary to a pandas dataframe

AEPm = [float(i) for i in list(mean_curve.index)] #Extract the annual exceedance probability

AEPmz = zvar(AEPm) #Convert to the normal z variate

Qmean = list(np.log10(mean_curve.Q_Mean_cfs)) #Extract the corresponding mean flow frequency discharge

Qmedian = list(np.log10(mean_curve.Q_Median_cfs)) #Extract the corresponding median flow frequency discharge

#### Calculate the discharge and add in the weights column:

In [5]:
table['Q_Mean_cfs'] = 10**table['AEPz'].apply(mean_q(AEPmz, Qmean)) #The discharge from the mean flow frequency curve

table['Q_Median_cfs'] = 10**table['AEPz'].apply(mean_q(AEPmz, Qmedian)) #The discharge from the median flow frequency curve

table = AEP_Weights.join(table, on = 'AEP')

table.head(2)

,AEP,Weight,AEPz,RI,Q_Mean_cfs,Q_Median_cfs
E0001,0.482369,0.035594,0.044208,2.073100,4005.034788,4019.950367
E0002,0.448352,0.032774,0.129827,2.230391,4195.386967,4208.765812


#### Save the results:

In [6]:
PRL = table.copy(deep=True).drop(columns=['AEPz']) #Drop the normal z variate

PRL.to_csv(pl.Path(root_dir)/gage_ID/'ProductionRunList_{}.csv'.format(gage_ID)) #Save the results as a csv

sb.glue('ProRunList', PRL.to_dict())

PRL.head(2)

,AEP,Weight,RI,Q_Mean_cfs,Q_Median_cfs
E0001,0.482369,0.035594,2.073100,4005.034788,4019.950367
E0002,0.448352,0.032774,2.230391,4195.386967,4208.765812


# End